Steam Store Games (Clean dataset)

Carga de todos los archivos en Google Colab

In [ ]:
import csv

input_file = "steam_description_data.csv"
output_file = "steam_description_fixed.csv"

with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8", newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    bad_rows = 0
    for idx, row in enumerate(reader):
        try:
            writer.writerow(row)
        except Exception as e:
            print(f"Fila problemática omitida en #{idx + 1}: {row}")
            bad_rows += 1

print(f"¡Listo! Se omitieron {bad_rows} filas problemáticas.")

1. Limpieza de df_main

In [ ]:
# Copia del dataframe original para mantener una versión limpia
df = df_main.copy()

# Mostrar estructura inicial
print("Resumen inicial:")
print(df.info())
print(df.head())

Revisión de columnas

In [ ]:
# Eliminar columnas que no aportan al análisis
columnas_a_eliminar = [
    "url", "reviews_url", "support_url", "store_url",
    "header_image", "website", "support_email"
]

df.drop(columns=columnas_a_eliminar, inplace=True, errors='ignore')

# Mostrar las columnas restantes
print("\nColumnas después de eliminar:")
print(df.columns)

Manejo de valores nulos

In [ ]:
# Revisar nulos por columna
print("\nValores nulos:")
print(df.isnull().sum())

# Eliminar filas con valores nulos en columnas clave
df = df.dropna(subset=["name", "release_date", "price", "categories", "genres"])

# Convertir fecha y precio a tipos correctos
df["release_date"] = pd.to_datetime(df["release_date"], errors='coerce')
df["price"] = pd.to_numeric(df["price"], errors='coerce')

# Eliminar juegos sin fecha válida o con precio negativo
df = df[df["release_date"].notnull() & (df["price"] >= 0)]

# Añadir columna de año
df["release_year"] = df["release_date"].dt.year

# Resultado final de limpieza
print("\nDespués de limpiar:")
print(df.info())

2. Análisis Exploratorio de Datos (EDA)

📊 Estadísticas generales

In [ ]:
# Estadísticas numéricas
print(df.describe())

# Número de juegos por año
print("\nJuegos por año:")
print(df["release_year"].value_counts().sort_index())

Análisis de géneros

In [ ]:
# Dividir géneros por coma y contar
from collections import Counter

generos_split = df["genres"].dropna().apply(lambda x: x.split(", "))
generos_flat = [g for sublist in generos_split for g in sublist]
conteo_generos = pd.Series(Counter(generos_flat)).sort_values(ascending=False)

print("\nTop 10 géneros:")
print(conteo_generos.head(10))

Visualizaciones

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Histograma de precios
plt.figure(figsize=(10,6))
df["price"].hist(bins=50, color="skyblue")
plt.title("Distribución de precios de juegos en Steam")
plt.xlabel("Precio (USD)")
plt.ylabel("Número de juegos")
plt.show()

# Juegos lanzados por año
plt.figure(figsize=(12,6))
df["release_year"].value_counts().sort_index().plot(kind="bar", color="orange")
plt.title("Juegos lanzados por año")
plt.xlabel("Año")
plt.ylabel("Cantidad de juegos")
plt.show()

# Gráfico de barras de géneros
plt.figure(figsize=(12,6))
conteo_generos.head(10).plot(kind="bar", color="teal")
plt.title("Top 10 géneros más comunes")
plt.xlabel("Género")
plt.ylabel("Número de juegos")
plt.show()

1. Cargar y limpiar steam.csv

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Cargar datos
df = pd.read_csv("steam.csv")

# Eliminar columnas innecesarias
columnas_a_eliminar = [
    "url", "reviews_url", "support_url", "store_url",
    "header_image", "website", "support_email"
]
df.drop(columns=columnas_a_eliminar, inplace=True, errors="ignore")

# Limpieza básica
df = df.dropna(subset=["name", "release_date", "price", "categories", "genres"])
df["release_date"] = pd.to_datetime(df["release_date"], errors='coerce')
df["price"] = pd.to_numeric(df["price"], errors='coerce')
df = df[df["release_date"].notnull() & (df["price"] >= 0)]
df["release_year"] = df["release_date"].dt.year

2. EDA y visualizaciones

In [ ]:
# Estadísticas
print(df.describe())

# Conteo de juegos por año
juegos_por_anio = df["release_year"].value_counts().sort_index()

# Conteo de géneros
generos_split = df["genres"].apply(lambda x: x.split(", "))
generos_flat = [g for sublist in generos_split for g in sublist]
conteo_generos = pd.Series(Counter(generos_flat)).sort_values(ascending=False)

Gráficos

In [ ]:
# Histograma de precios
plt.figure(figsize=(10,5))
df["price"].hist(bins=50, color='skyblue')
plt.title("Distribución de precios de juegos en Steam")
plt.xlabel("Precio (USD)")
plt.ylabel("Número de juegos")
plt.show()

# Juegos lanzados por año
plt.figure(figsize=(12,5))
juegos_por_anio.plot(kind="bar", color="orange")
plt.title("Número de juegos lanzados por año")
plt.xlabel("Año")
plt.ylabel("Cantidad")
plt.show()

# Top 10 géneros
plt.figure(figsize=(10,5))
conteo_generos.head(10).plot(kind="bar", color="teal")
plt.title("Top 10 géneros más comunes en Steam")
plt.xlabel("Género")
plt.ylabel("Frecuencia")
plt.show()

3. Unión con steamspy_tag_data.csv

In [ ]:
print(df_tags.columns)
print(df_tags.head())

In [ ]:
columnas_usar = []
for nombre in ["name", "release_year", "price", "Action", "Adventure", "Indie"]:
    if nombre in df_merged.columns:
        columnas_usar.append(nombre)

print(df_merged[columnas_usar].head())

In [ ]:
for col in df_tags.columns:
    print(col)

In [ ]:
relevantes = [col for col in df_tags.columns if any(tag in col.lower() for tag in ["action", "adventure", "indie"])]
print("Columnas relevantes encontradas:", relevantes)

In [ ]:
print(df_merged[["name", "release_year", "price", "action", "adventure", "indie"]].head())

In [ ]:
# Juegos que SÍ tienen la tag de acción en cualquier cantidad
accion_por_anio = df_merged[df_merged[accion_col] > 0]["release_year"].value_counts().sort_index()

In [ ]:
accion_col = 'action'  # Ya detectado como relevante
accion_por_anio = df_merged[df_merged[accion_col] > 0]["release_year"].value_counts().sort_index()

import matplotlib.pyplot as plt
plt.figure(figsize=(12,5))
accion_por_anio.plot(kind="bar", color="crimson")
plt.title("Juegos de Acción lanzados por año")
plt.xlabel("Año")
plt.ylabel("Cantidad")
plt.show()

4. Conclusiones clave

# Conclusiones

- La mayoría de los juegos en Steam tienen precios entre $0 y $20.
- Los años con más lanzamientos fueron 2017 y 2018.
- Los géneros más frecuentes son: **Action**, **Indie**, y **Adventure**.
- Los juegos de tipo Acción han tenido un crecimiento significativo desde 2014.
- El dataset fusionado con tags nos permite hacer análisis por popularidad de género.

5. README.md para GitHub
markdown
Copiar
Editar
# 🎮 Análisis de Juegos en Steam

Este proyecto analiza datos de más de 25,000 videojuegos publicados en Steam. Se utilizaron múltiples archivos CSV para obtener una visión completa del ecosistema de juegos, precios, géneros y tendencias.

## Archivos utilizados

- `steam.csv` (datos principales de cada juego)
- `steamspy_tag_data.csv` (tags como Action, Indie, etc.)
- Otros archivos para futura expansión: descripciones, requisitos, soporte.

## Análisis realizado

- Limpieza y transformación de datos
- Análisis exploratorio de precios, años de lanzamiento y géneros
- Visualización de datos con Matplotlib y Seaborn
- Unión con tags populares por género

## Resultados destacados

- Género más común: **Action**
- La mayoría de los juegos cuestan entre $0 y $20
- Gran cantidad de lanzamientos a partir de 2014
- Crecimiento constante de juegos Indie

## Herramientas

- Python 3.10
- Pandas, Matplotlib, Seaborn
- Google Colab

## Conclusión

Steam es una plataforma diversa con fuerte presencia de juegos Indie y de bajo precio. El análisis de tags muestra la evolución de géneros como Acción y Aventura, y la alta concentración de lanzamientos desde 2014 en adelante.